######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

# ESML - accelerator: Batch scoring pipeline. 6 acceleration benefits 
- 1) `AutoMap datalake` & init ESML project
- 2) `Get correct environment` - via ESML config, to the correct Workspace(dev,test,prod)
- 3) `1 line: Get Compute cluster` from ESML.get_training_aml_compute 
- 4) `1 line: Get earlier trained model` and `Inference ENVIRONMENT` from AutoML via ESMLProject.get_active_model_inference_config()
- 5) `DATASET via properties:  `p.DatasetByName("ds01_diabetes").Bronze `and `p.GoldToScore`  `and `p.GoldScored`
    - Or via conventions name from portal `M11_ds02_other_inference_BRONZE` and `M11_GOLD_TO_SCORE` and  `M11_GOLD_SCORED`
- 6) `Score and Writeback`: Save to scored data to "somewhere". Example: riteBack functionality to source

# USAGE:
You can use ESMLPipeline factory like this notebook:
`ESMLPipeline factory will build the pipeline automatically`, all steps based on the dataset array in the `model_settings.json` and witht the `ESML Datamodel: Bronze->Silver-Gold` 

## THIS CODE... ↓
```
p = ESMLProject()
p_factory = ESMLPipelineFactory(p, "Y")

p_factory.create_dataset_scripts_from_template() # Do this once, then edit them manually
batch_pipeline = p_factory.create_batch_scoring_pipe()
pipeline_run = p_factory.execute_pipeline(batch_pipeline)

pipeline_run.wait_for_completion(show_output=False)
```

## ...WILL GIVE YOU THAT PIPELINE ↓ (Note: Datastore is Azure Datlake GEN2 : )
You will get an image like this, if you have 2 datasets in lake_settings.json

![](../azure-enterprise-scale-ml/esml/images/pipeline_IN2GOLD.png)

- Above PIPELINE: "dataset_folder_names": ["ds01_diabetes", "ds02_other"]
- You can edit each `step-file.py` which is generate by ESML

# 1) `One cell of code: 3 lines` below, to create above, and execute it

In [2]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types
p = ESMLProject()

p.active_model = 11 # Y=11, price=12
p_factory = ESMLPipelineFactory(p)

#scoring_date = '2021-01-01 10:35:01.243860'
#p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.

p.inference_mode = True
p_factory.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
File to EDIT a lot (reference in step-scripts Custom code): ../../../2_A_aml_pipeline/4_inference/batch/M11/your_code/your_custom_code.py

 ---- WHAT model to SCORE with, & WHAT data 'date_folder'? ---- 
InferenceModelVersion (model version to score with): 1
Date_scoring_folder (data to score) : 2021-06-08 15:35:01.243860
ESML environment: dev

 ---- ESML Datalake locations: ES

## 1a) - BUILD & `RUN` pipeline (3 lines)
- `Iterate` until you are happy with the pipeline (edit step files etc)

In [3]:
## BUILD IN_2_GOLD_SCORING
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING) # Creates pipeline from template

Creates template step_files.py for user to edit at:
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/your_code/your_custom_code.py


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Using GEN2 as Datastore
found model via REMOTE FILTER: Experiment TAGS: model name and version
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster prj02-m11-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = prj02-m11-dev


In [7]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Created step IN 2 SILVER - ds01_diabetes [dbed732f][f368c343-3f6b-4e6a-8d3f-027e127aab17], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_other [a3aea9a1][6585b8e7-42af-4f09-848c-7dbb82eef202], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [74eda51d][c0feb0a7-5842-4554-86b4-2ca403bd2f51], (This step will run and generate new outputs)
Created step SCORING GOLD [a1eaf671][0caf5173-d9d8-441e-ba6a-6a228b080fe9], (This step will run and generate new outputs)
Submitted PipelineRun 6ffc4b12-5266-4d1b-af93-056f652cc248
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6ffc4b12-5266-4d1b-af93-056f652cc248?wsid=/subscriptions/ca0a8c40-b06a-4e4e-8434-63c03a1dee34/resourcegroups/MSFT-WEU-EAP_PROJECT02_AI-DEV-RG/workspaces/msft-weu-DEV-eap-proj02_ai-amls&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline submitted for execution!
PipelineRunId: 6ffc4b12-5266-4d1b-af93-056f652cc248
Link to Azure Machine Learning Portal: ht

'Finished'

## 1b) When satisfied - `PUBLISH` pipeline (or rebuild and publish)

In [2]:
# REBUILD - if you haven't runned the above cell, uncommen below:
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # overwrite_if_exists=False is default
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING) # Gets workspace, connects to lake, creates pipeline.
p.ws = p.get_workspace_from_config()

# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline, "_4") # "_4" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Loading AutoML config settings from: dev
Loading AutoML config settings from: dev


Package:azureml-automl-runtime, training version:1.30.0, current version:1.26.0
Package:azureml-core, training version:1.30.0, current version:1.26.0
Package:azureml-dataprep, training version:2.15.1, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.13.0, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.30.0, current version:1.26.0
Package:azureml-defaults, training version:1.30.0, current version:1.26.0
Package:azureml-interpret, training version:1.30.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.30.0, current version:1.26.0
Package:azureml-telemetry, training version:1.30.0, current version:1.26.0
Package:azureml-train-automl-client, training version:1.30.0, current version:1.26.0
Package:azureml-train-automl-runtime, training version:1.30.0, current version:1.26.0


Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster prj02-m11-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = prj02-m11-dev
Created step IN 2 SILVER - ds01_diabetes [765b1b27][59ee5823-5533-47ef-8954-6db1cb9667dc], (This step is eligible to reuse a previous run's output)Created step IN 2 SILVER - ds02_other [d00e55da][03310cbe-a12e-4a22-a88b-141c5731f12e], (This step is eligible to reuse a previous run's output)

Created step SILVER MERGED 2 GOLD [1c8c506d][48b865c8-282d-40f8-8008-bbef4ef1e1b4], (This step is eligible to reuse a previous run's output)
Created step SCORING GOL

# Another pipeline-type - IN_2_GOLD

In [ ]:
## BUILD IN_2_GOLD_SCORING
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

# 2) `CONSUME` pipeline: HowTo

## 2a) Consume from `Azure Data factory - BATCH_SCORE Pipeline activity`

In [14]:
print("2 parameters needed to be set, to call PIPELINE activity") 
print("")
print("esml_inference_model_version=0 ")
print(" - 0 means use latest version, but you can pick whatever version in the DROPDOWN in Azure data factory you want")
print("esml_scoring_folder_date='2021-06-08 15:35:01.243860'")
print(" - DateTime in UTC format. Example: For daily scoring 'datetime.datetime.now()'")

2 parameters needed to be set, to call PIPELINE activity

esml_inference_model_version=0 
 - 0 means use latest version, but you can pick whatever version in the DROPDOWN in Azure data factory you want
esml_scoring_folder_date='2021-06-08 15:35:01.243860'
 - DateTime in UTC format. Example: For daily scoring 'datetime.datetime.now()'


In [ ]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

2) Fetch scored data: Below needed for Azure Data factory WRITEBACK activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  7659731b-2e34-452a-babe-8149ecd15b70
Endpoint Name:  11_diabetes_model_reg_batch_scoring_pipe_EP_3
Experiment name:  11_diabetes_model_reg_batch_scoring_pipe


## 2b) Consume from `Azure Data factory - WriteBack Pipeline activity`

In [28]:
from azureml.core.dataset import Dataset
from azureml.core import Experiment
from  azureml.pipeline.core import PipelineRun

# 1st you need a "Post scoring" activity, to get metadata of "scored_gold_path" from "last_gold_run.csv"
ds1 = Dataset.get_by_name(workspace = p.ws, name =  p.dataset_gold_scored_runinfo_name_azure)
run_id = ds1.to_pandas_dataframe().iloc[0]["pipeline_run_id"] # ['pipeline_run_id', 'scored_gold_path', 'date_in_parameter', 'date_at_pipeline_run','model_version'])
scored_gold_path = ds1.to_pandas_dataframe().iloc[0]["scored_gold_path"]

print("Read this meta-dataset from ADF: {}/last_gold_run.csv".format(p.path_inference_gold_scored_runinfo))
print("- To get the column 'scored_gold_path' which points to the scored-data:")
print("{}*.parquet".format(scored_gold_path))


Read this meta-dataset from ADF: projects/project002/11_diabetes_model_reg/inference/active/gold_scored_runinfo/last_gold_run.csv
- To get the column 'scored_gold_path' which points to the scored-data:
projects/project002/11_diabetes_model_reg/inference/1/scored/dev/2021/06/08/21bf4e04-df7b-4c3a-a8f8-f3b32bf3a02e/*.parquet


## 2b) Consume from `from PYTHON`
- Run a pipeline endpoint (`Python SDK` call)

In [15]:
from azureml.pipeline.core import PipelineEndpoint
pipeline_endpoint = PipelineEndpoint.get(workspace=p.ws, name=p_factory.name_batch_pipeline_endpoint)
pipeline_run_sdk = pipeline_endpoint.submit(p_factory.experiment_name)
pipeline_run_sdk.id

In [22]:
pipeline_run_sdk.status

'Running'

## 2c) Consume from `from PYTHON`
- Run via REST call

In [18]:
from azureml.pipeline.core import PublishedPipeline,PipelineEndpoint,PipelineRun
import requests
from azureml.core.authentication import ServicePrincipalAuthentication # InteractiveLoginAuthentication, AzureCliAuthentication

sp = p.get_authenticaion_header_sp()
auth_header = sp.get_authentication_header()
date_folder = str(p.date_scoring_folder)
pipeline_endpoint = PipelineEndpoint.get(workspace=p.ws, name=p_factory.name_batch_pipeline_endpoint)

response = requests.post(pipeline_endpoint.endpoint,
                         headers=auth_header,
                         json={"ExperimentName": p_factory.experiment_name,
                               "ParameterAssignments": {
                                     "esml_inference_model_version": p.inferenceModelVersion,
                                     "esml_scoring_folder_date": date_folder
                                     }
                              })

In [19]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  ebfd7102-e8af-47ab-a255-97b9ce334e4e


### View status from REST call, via SDK

In [20]:
from azureml.pipeline.core import PublishedPipeline,PipelineEndpoint,PipelineRun
published_pipeline_run = PipelineRun(p.ws.experiments[p_factory.experiment_name], run_id)

In [23]:
published_pipeline_run.status

'Running'

# `WHO is the caller, usually?` (Azure Data factory, Azure Devops)` - that sends PARAMETERS and WHY?` 
### Q: Why? 
- A: To use same DEV scoring pipeline, with either different data to be scored `daily scoring`, or `different model-version SAME day` to score with.
- A: To have "environment parameters (dev,test,prod) we can instatiate a ESMLProject what knows the lake, workspace, makes it easy to create 3 pipelines for dev,test,prod
    - And data, if 1 LAKE or 3 LAKES (dev,test,prod), they all have data-folders "dev,test,prod"

### Who gives input?
- A) Azure Devops (CI/CD) will trigger TRAIN pipeline, that will end with creating this BATCH SCORING, with 
    - 2 parameters (`esml_environment, esml_inference_model_version`), to CREATE/UPDATE the BATCH pipeline with newly trained model
    - 1 dummy (`esml_scoring_folder_date`) to test BATCH SCORING after creation.
- B) Azure Datafactory (read from source, writes as .csv or .parquet to IN-folder), and will trigger BATCH SCORING with:
    - 2 PIPELINE parameters (`esml_inference_model_version, esml_scoring_folder_date`), to read IN-DATA to be scored. Usually "todays" esml_scoring_folder_date
    - Note: To solve "many scorings same day", a "run.id" folder is created before the actual data.parquet
    - Note: `*esml_environment` is not really needed post creation - since we already created the pipleine in DEV, `locked and loaded`

### Who needs `scored_data` and HOW to get it? META data:
- `Azure Datafactory` can read meta data of `last scored GOLD`, to get datalake-path of SCORED_GOLD - can then "`write back scored data`" to source, or another `system`
    - See next cells "`Get previous RUN and PIPELINE via `ESML` metadata`"
- `Power BI` can read the meta-data to fetch `last scored GOLD` directly

## Get previous RUN and PIPELINE via `ESML` metadata
- How to get path of `scored_gold_path` and how to see the actual `pipeline run`

In [6]:
from azureml.core.dataset import Dataset
from azureml.core import Experiment
from  azureml.pipeline.core import PipelineRun

# Get "Pipeline run" info, for tghe most recent "latest scored gold"
ds1 = Dataset.get_by_name(workspace = p.ws, name =  p.dataset_gold_scored_runinfo_name_azure)
run_id = ds1.to_pandas_dataframe().iloc[0]["pipeline_run_id"] # ['pipeline_run_id', 'scored_gold_path', 'date_in_parameter', 'date_at_pipeline_run','model_version'])
scored_gold_path = ds1.to_pandas_dataframe().iloc[0]["scored_gold_path"]

print("dataset_gold_scored_runinfo, location: {}".format)
print("pipeline_run_id: {}".format(run_id))
print("scored_gold_path: '{}'".format(scored_gold_path))

experiment = Experiment(workspace=p.ws, name=p_factory.experiment_name)
remote_run = PipelineRun(experiment=experiment, run_id=run_id)
print("\nFetched RUN object {}".format(remote_run))

pipeline_run_id: 0a7a6ad1-5493-4855-9cab-342ae7da29f0
scored_gold_path: 'projects/project002/11_diabetes_model_reg/inference/1/scored/dev/2021/06/08/0a7a6ad1-5493-4855-9cab-342ae7da29f0/'

Fetched RUN object Run(Experiment: 11_diabetes_model_reg_batch_scoring_pipe,
Id: 0a7a6ad1-5493-4855-9cab-342ae7da29f0,
Type: azureml.PipelineRun,
Status: Completed)


# `What can you configure?` (parameters, step compute, custom code)
## 1) Configure: Parameters
- Pipeline parameters: scoring_date, model_version
    - Why: To dynamically select different data & model to score with, with same pipeline/reuse.
    - Who: Azure data factory can dynamically set these, and call AML pipline
- Pipeline parameters (model specific): target_column_name
    - Why: To merge datasets to GOLD.
print("Model version (pipeline parameter): {}".format(p_factory.batch_pipeline_parameters[0].default_value))
print(" - This default value is set from ESMLProject settings: {}".format(p.inferenceModelVersion))
print("Scoring datetime: {}".format(p_factory.batch_pipeline_parameters[1].default_value))
print(" - This default value is set from ESMLProject settings: {}".format(str(p.date_scoring_folder)))
# Optional parameters to READ or SET
#parameters[2].name: parameters[2].default_value, # esml_optional_unique_scoring_folder 
#parameters[3].name: parameters[3].default_value # par_esml_dev_test_prod
## 2) Configure: Compute & Environment (via ESML config or inject your own)
- `Different compute per step OR samee for all` ["cpu","gpu", "databricks"], based on your ESML environment (dev,test,prod) compute settings, and Dataset properties.
 - A) `Different compute for all steps`

        -  if(dataset.cpu_gpu_databricks == "cpu"):
        -       compute, runconfig = self.init_cpu_environment()
        -  elif(d.cpu_gpu_databricks == "databricks"):
        -     compute, runconfig = self.init_databricks_environment()
        -  elif(d.cpu_gpu_databricks == "gpu"):
        -       compute, runconfig = self.init_gpu_environment()
- B) `Same compute for all`: For the full pipeline, is the DEFAULT behaviour.

        - def `create_batch_scoring_pipe(self, `same_compute_for_all=True`, `cpu_gpu_databricks="cpu")`


# DDL "IN" and "WriteBack" table: SQL Server
 - Tables to create for WriteBack demo

```sql
-- 1) IN DATA to Lake, anonymized
CREATE TABLE [dbo].[esml_diabetes]
(
    -- PersonId: Not needed for ML scoring. It is actually only noise for the Machine Learning brain. 
    PersonId INT IDENTITY(1,1) not null, -- But IF we want to reconnect scored RESULT to an individual, we need it.
	AGE FLOAT NOT NULL,
	SEX FLOAT NOT NULL,
	BMI FLOAT NOT NULL,
	BP FLOAT NOT NULL,
	S1 FLOAT NOT NULL,
	S2 FLOAT NOT NULL,
	S3 FLOAT NOT NULL,
	S4 FLOAT NOT NULL,
	S5 FLOAT NOT NULL,
	S6 FLOAT NOT NULL
)

-- 2) Scored data the PIPELINE WroteBack
CREATE TABLE [dbo].[esml_personID_scoring]
(
    PersonId INT NOT NULL,
    DiabetesMLScoring DECIMAL NULL,
    scoring_time DATETIME NULL,
    in_data_time DATETIME NULL,
    ts DATETIME NOT NULL DEFAULT (GETDATE())
)
-- SELECT Count(*) as total_rows FROM [dbo].[esml_personID_scoring] -- 442 rows per RUN since "UPSERT" from Azure Datafactory on PersonID
-- SELECT * FROM [dbo].[esml_personID_scoring]

-- 3) VIEW Person connected to scoring: Risk of DIABETES

--SELECT * FROM [dbo].[esml_diabetes] as a
SELECT * FROM [dbo].[esml_person_info] as a
LEFT JOIN [dbo].[esml_personID_scoring] as b
ON a.PersonId = b.PersonId

```